# Exercise 04 : A/B-testing

In [65]:
import pandas as pd
import sqlite3

In [66]:
con = sqlite3.connect('../data/checking-logs.sqlite')

In [67]:
test_results = pd.read_sql("""SELECT CASE WHEN first_commit_ts < first_view_ts THEN 'after' ELSE 'before' END AS time,
  AVG((deadlines - strftime('%s', first_commit_ts)) / 3600.0) AS avg_diff
                              FROM test
                                  JOIN deadlines
                              ON test.labname = deadlines.labs
                              WHERE labname != 'project1'
                                AND uid IN (
                                  SELECT uid FROM test
                                  WHERE labname != 'project1'
                                  GROUP BY uid
                                  HAVING
                                  SUM (first_commit_ts
                                  < first_view_ts)
                                  > 0
                                AND
                                  SUM (first_commit_ts
                                  > first_view_ts)
                                  > 0
                                  )
                              GROUP BY time;
                           """
                           , con)

In [68]:
test_results

,time,avg_diff
0,after,61.156632
1,before,105.229241


In [69]:
control_results = pd.read_sql("""
                              SELECT CASE WHEN first_commit_ts < first_view_ts THEN 'after' ELSE 'before' END AS time,
  AVG((deadlines - strftime('%s', first_commit_ts)) / 3600.0) AS avg_diff
                              FROM control
                                  JOIN deadlines
                              ON control.labname = deadlines.labs
                              WHERE labname != 'project1'
                                AND uid IN (
                                  SELECT uid FROM control
                                  WHERE labname != 'project1'
                                  GROUP BY uid
                                  HAVING
                                  SUM (first_commit_ts
                                  < first_view_ts)
                                  > 0
                                AND
                                  SUM (first_commit_ts
                                  > first_view_ts)
                                  > 0
                                  )
                              GROUP BY time;

                              """
                              , con)

In [70]:
control_results

,time,avg_diff
0,after,99.901448
1,before,118.144571


In [71]:
comparison = pd.merge(test_results, control_results, on="time")

In [72]:
comparison

,time,avg_diff_x,avg_diff_y
0,after,61.156632,99.901448
1,before,105.229241,118.144571


## Гипотеза верная, страница влияет на поведение студентов